### Configuration of the environment

In [ ]:
%tensorflow_version 2.x
!pip3 install --upgrade pip
#!pip install -qU t5
!pip3 install git+https://github.com/google-research/text-to-text-transfer-transformer.git #extra_id_x support

import functools
import os
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds

import t5

#Set the base dir(Google cloud bucket)
BASE_DIR = "gs://bucket_code_completion" 

if not BASE_DIR or BASE_DIR == "gs://":
  raise ValueError("You must enter a BASE_DIR.")
ON_CLOUD = True


if ON_CLOUD:
  import tensorflow_gcs_config
  from google.colab import auth
  # Set credentials for GCS reading/writing from Colab and TPU.
  TPU_TOPOLOGY = "2x2"
  try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    TPU_ADDRESS = tpu.get_master()
    print('Running on TPU:', TPU_ADDRESS)
  except ValueError:
    raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')
  auth.authenticate_user()
  tf.config.experimental_connect_to_host(TPU_ADDRESS)
  tensorflow_gcs_config.configure_gcs_from_colab_auth()

tf.disable_v2_behavior()

# Improve logging.
from contextlib import contextmanager
import logging as py_logging

if ON_CLOUD:
  tf.get_logger().propagate = False
  py_logging.root.setLevel('INFO')

@contextmanager
def tf_verbosity_level(level):
  og_level = tf.logging.get_verbosity()
  tf.logging.set_verbosity(level)
  yield
  tf.logging.set_verbosity(og_level)

     |████████████████████████████████| 1.5MB 7.0MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
  Cloning https://github.com/google-research/text-to-text-transfer-transformer.git to /tmp/pip-req-build-zrulez44
  Running command git clone -q https://github.com/google-research/text-to-text-transfer-transformer.git /tmp/pip-req-build-zrulez44
     |████████████████████████████████| 361 kB 5.3 MB/s 
     |████████████████████████████████| 65 kB 3.2 MB/s 
     |████████████████████████████████| 1.2 MB 9.8 MB/s 
     |████████████████████████████████| 3.4 MB 12.7 MB/s 
     |████████████████████████████████| 3.8 MB 64.7 MB/s 
     |████████████████████████████████| 1.8 MB 82.9 MB/s 
     |████████████████████████████████| 2.9 MB 86.7 MB/s 
     |████████████████████████████████| 883 kB 84.7 MB/s 
  Created wheel for t5: filename=t5-0.8.1-py3-none-any.whl size=220069 sha256=d7c5ddf0186d9b0d510e9ff3799b01c85cd1d17bd0dcf9f5

Instructions for updating:
non-resource variables are not supported in the long term


### Loading of the tsv files
We loaded the 6 tsv files, please be sure to upload them on the bucket and to copy the correct path

In [ ]:
#Validation(train and test on the same dataset)

nq_tsv_path_java_construct = {
    "train":      'gs://bucket_code_completion/T5_extension/ft_datasets/train_java_construct.tsv',
    "validation": 'gs://bucket_code_completion/T5_extension/ft_datasets/eval_java_construct.tsv',
}

num_nq_examples_java_construct = dict(train=750000, validation=104037)

In [ ]:
#Validation(train and test on the same dataset)

nq_tsv_path_android_construct = {
    "train":      'gs://bucket_code_completion/T5_extension/ft_datasets/train_android_construct.tsv',
    "validation": 'gs://bucket_code_completion/T5_extension/ft_datasets/eval_android_construct.tsv',
}

num_nq_examples_android_construct = dict(train=750000, validation=98793)

In [ ]:
#Validation(train and test on the same dataset)

nq_tsv_path_java_block = {
    "train":      'gs://bucket_code_completion/T5_extension/ft_datasets/train_java_block.tsv',
    "validation": 'gs://bucket_code_completion/T5_extension/ft_datasets/eval_java_block.tsv',
}

num_nq_examples_java_block = dict(train=298470, validation=38840)

In [ ]:
#Validation(train and test on the same dataset)

nq_tsv_path_android_block = {
    "train":      'gs://bucket_code_completion/T5_extension/ft_datasets/train_android_block.tsv',
    "validation": 'gs://bucket_code_completion/T5_extension/ft_datasets/eval_android_block.tsv',
}

num_nq_examples_android_block = dict(train=204580, validation=26503)

In [ ]:
#Validation(train and test on the same dataset)

nq_tsv_path_java_token = {
    "train":      'gs://bucket_code_completion/T5_extension/ft_datasets/train_java_token.tsv',
    "validation": 'gs://bucket_code_completion/T5_extension/ft_datasets/eval_java_token.tsv',
}

num_nq_examples_java_token = dict(train=750000, validation=214682)

In [ ]:
#Validation(train and test on the same dataset)

nq_tsv_path_android_token = {
    "train":      'gs://bucket_code_completion/T5_extension/ft_datasets/train_android_token.tsv',
    "validation": 'gs://bucket_code_completion/T5_extension/ft_datasets/eval_android_token.tsv',
}

num_nq_examples_android_token = dict(train=750000, validation=198281)

### Preprocess of the dataset
In this step we preprocess the dataset.  
You have to change the path to vocab files (*vocab_model_path* and *vocab_path*)
We're going to preprocess all the tsv file so that T5 can use them for HP tuning.

In [ ]:
from t5.data import postprocessors as t5_postprocessors
from t5.seqio import Feature,SentencePieceVocabulary


# # Set the path of sentencepiece model and vocab files
# # Must be the same used for the pre-trained phase
vocab_model_path = 'gs://bucket_code_completion/T5_extension/code.model'
vocab_path = 'gs://bucket_code_completion/T5_extension/code.vocab'


TaskRegistry = t5.data.TaskRegistry
TfdsTask = t5.data.TfdsTask


def get_default_vocabulary():
  return SentencePieceVocabulary(vocab_model_path, 100)

DEFAULT_OUTPUT_FEATURES = {
    "inputs": Feature(
        vocabulary=get_default_vocabulary(), add_eos=True, required=False),

    "targets": Feature(
        vocabulary=get_default_vocabulary(), add_eos=True)
}

JAVA CONSTRUCT

In [ ]:
def nq_java_construct(split, shuffle_files=True):
  # We only have one file for each split.
  del shuffle_files

   # Load lines from the text file as examples.

  ds = tf.data.TextLineDataset(nq_tsv_path_java_construct[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

print("A few raw train examples...")
for ex in tfds.as_numpy(nq_java_construct("train").take(5)):
  print(ex)

A few raw train examples...
{'input': b'public void updateLockdownExceptions(ManagedObjectReference _this, String[] users) throws AuthMinimumAdminPermission, RemoteException, RuntimeFault, UserNotFound { Argument[] params = new Argument[2]; params[0] = new Argument("_this", "ManagedObjectReference", _this); params[1] = new Argument("users", "String[]", users); getWsc().invoke( <extra_id_0>); }', 'output': b'"UpdateLockdownExceptions", params, null'}
{'input': b'@Override public Collection<AgentProjectInfo> collectDependencies(String folder) { if ( <extra_id_0>){ devDependencies = findDevDependencies(folder); } File yarnLock = new File(folder + fileSeparator + YARN_LOCK); boolean yarnLockFound = yarnLock.isFile(); Collection<DependencyInfo> dependencies = new ArrayList<>(); if (yarnLockFound){ dependencies = parseYarnLock(yarnLock); } else { npmLsFailureStatus = true; } return getSingleProjectList(dependencies); }', 'output': b'!includeDevDependencies'}
{'input': b'@Override public Coll

In [ ]:
def java_construct_preprocessing(ds):
  
  def to_inputs_and_targets(ex):

        inputs = tf.strings.join(['JAVA_CONSTRUCT:' + ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label }
    
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
t5.data.TaskRegistry.remove('java_construct')
t5.data.TaskRegistry.add(
    "java_construct",
    dataset_fn=nq_java_construct,
    splits=["train", "validation"],
    text_preprocessor=[java_construct_preprocessing],
    output_features = DEFAULT_OUTPUT_FEATURES,
    metric_fns=[t5.evaluation.metrics.accuracy],
    num_input_examples=num_nq_examples_java_construct
)

In [ ]:
nq_task = t5.data.TaskRegistry.get("java_construct")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": 256, "targets": 256})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)


/usr/local/lib/python3.6/dist-packages/t5/seqio/preprocessors.py:65: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  _tokenize, num_parallel_calls=tf.data.experimental.AUTOTUNE)


A few preprocessed training examples...
{'inputs_pretokenized': b'JAVA_CONSTRUCT:@Override public Object toJdbc(final Optional<?> original, final Connection connection, final BindParameterMapperManager parameterMapperManager) { return original.isPresent() ? parameterMapperManager.toJdbc( <extra_id_0>) : null; }', 'inputs': array([    3,  7641,    15,  5071, 17558,    56,  2098,    27,    12,
         102,    81,  5537,     5,    64,   730,    25,     2,    29,
        1302,     9,    44,  2097,   557,     9,    44,     3,  3003,
         513,   898,   121,  1218,   898,   121,     8,     7,    14,
        1302,     4,  2610,    16,     3,     2,  1218,   898,   121,
           4,   314,  5537,     5, 32099,     8,    58,    30,    13,
           6,     1], dtype=int32), 'targets_pretokenized': b'original.get(), connection', 'targets': array([1302,    4,   33,   72,  557,    1], dtype=int32)}
{'inputs_pretokenized': b'JAVA_CONSTRUCT:private void cleanUpDynamicPackages() throws IOExcepti

JAVA TOKEN

In [ ]:
def nq_java_token(split, shuffle_files=False):
  # We only have one file for each split.
  del shuffle_files

  # Load lines from the text file as examples.
  ds = tf.data.TextLineDataset(nq_tsv_path_java_token[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

print("A few raw valid examples...")
for ex in tfds.as_numpy(nq_java_token("validation").take(5)):
  print(ex)

A few raw valid examples...
{'input': b'public void addThirdPartyPaymentWorkflow(com.mozu.api.contracts.sitesettings.order.ExternalPaymentWorkflowDefinition definition) throws Exception { MozuClient client = com.mozu.api.clients.commerce.settings.checkout.PaymentSettingsClient <extra_id_0> client.setContext(_apiContext); client.executeRequest(); client.cleanupHttpConnection(); }', 'output': b'.addThirdPartyPaymentWorkflowClient( definition);'}
{'input': b'public void addThirdPartyPaymentWorkflow(com.mozu.api.contracts.sitesettings.order.ExternalPaymentWorkflowDefinition definition) throws Exception { MozuClient client = com.mozu.api.clients.commerce.settings.checkout.PaymentSettingsClient.addThirdPartyPaymentWorkflowClient( definition); client.setContext(_apiContext <extra_id_0> client.executeRequest(); client.cleanupHttpConnection(); }', 'output': b');'}
{'input': b'public void addThirdPartyPaymentWorkflow(com.mozu.api.contracts.sitesettings.order.ExternalPaymentWorkflowDefinition def

In [ ]:
def java_token_preprocessing(ds):
  
  def to_inputs_and_targets(ex):

        inputs = tf.strings.join(['JAVA_TOKEN:' + ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label }
    
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
t5.data.TaskRegistry.remove('java_token')
t5.data.TaskRegistry.add(
    "java_token",
    dataset_fn=nq_java_token,
    splits=["train", "validation"],
    text_preprocessor=[java_token_preprocessing],
    output_features = DEFAULT_OUTPUT_FEATURES,
    metric_fns=[t5.evaluation.metrics.accuracy],
    num_input_examples=num_nq_examples_java_token
)

In [ ]:
nq_task = t5.data.TaskRegistry.get("java_token")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": 256, "targets": 256})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)


/usr/local/lib/python3.6/dist-packages/t5/seqio/preprocessors.py:65: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  _tokenize, num_parallel_calls=tf.data.experimental.AUTOTUNE)


A few preprocessed training examples...
{'inputs_pretokenized': b'JAVA_TOKEN:public static void addMessage(String bundleBaseName, FacesMessage.Severity severity, String messageId, Object[] args, String forClientId, FacesContext facesContext) { if(log.isLoggable(Level.FINEST <extra_id_0> { log.finest("adding message " + messageId + " for clientId " + forClientId); } facesContext.addMessage(forClientId, getMessage(bundleBaseName, severity, messageId, args, facesContext)); }', 'inputs': array([    3,  7641,    15,  2591,    56,  4569,    48,    20,     3,
        9341,     5,    31,  1405,   431,    66,     9,     3, 12563,
         155,     4,  5015,  8007,     9,    26,  7862,     9,   102,
          61,   294,     9,    26,    50, 10414,     9,     3, 10003,
           3, 23090,     8,     7,    21,     5,   417,     4,  7417,
           5,   377,     4, 24151, 32099,     7,   224,     4, 18887,
          28,    67,   150,   175,    32,    34,  7862,    34,    32,
          50,  4938, 

JAVA BLOCK

In [ ]:
def nq_java_block(split, shuffle_files=False):
  # We only have one file for each split.
  del shuffle_files

  # Load lines from the text file as examples.
  ds = tf.data.TextLineDataset(nq_tsv_path_java_block[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

print("A few raw valid examples...")
for ex in tfds.as_numpy(nq_java_block("validation").take(5)):
  print(ex)

A few raw valid examples...
{'input': b'public Object createField(String name) { Object reader = _fieldMap.get(name); if (reader == null) <extra_id_0> return reader; }', 'output': b'reader = NullFieldDeserializer.DESER;'}
{'input': b'@Nonnull public static JSInvocation invoke (@Nonnull final JQueryInvocation aJQueryInvocation, @Nonnull final JSAssocArray aOptions) <extra_id_0>', 'output': b'{ return invoke (aJQueryInvocation).arg (aOptions); }'}
{'input': b'public static String presentMinMaxCount(long minmax) { if (minmax == Long.MAX_VALUE || minmax == Long.MIN_VALUE) <extra_id_0> return String.valueOf(minmax); }', 'output': b'{ return UNDEF_STRING; }'}
{'input': b'@Override public NonBottomTypeNode<ElkClass, ElkNamedIndividual> getCreateNode( final Collection<? extends ElkClass> members) <extra_id_0>', 'output': b'{ return getCreateUpdateableTypeNode( classTaxonomy_.getCreateNode(members)); }'}
{'input': b'public static JsonException typeMismatch(Object indexOrName, Object actual, Str

In [ ]:
def java_block_preprocessing(ds):
  
  def to_inputs_and_targets(ex):

        inputs = tf.strings.join(['JAVA_BLOCK:' + ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label }
    
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
t5.data.TaskRegistry.remove('java_block')
t5.data.TaskRegistry.add(
    "java_block",
    dataset_fn=nq_java_block,
    splits=["train", "validation"],
    text_preprocessor=[java_block_preprocessing],
    output_features = DEFAULT_OUTPUT_FEATURES,
    metric_fns=[t5.evaluation.metrics.accuracy],
    num_input_examples=num_nq_examples_java_block
)

In [ ]:
nq_task = t5.data.TaskRegistry.get("java_block")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": 256, "targets": 256})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)


/usr/local/lib/python3.6/dist-packages/t5/seqio/preprocessors.py:65: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  _tokenize, num_parallel_calls=tf.data.experimental.AUTOTUNE)


A few preprocessed training examples...
{'inputs_pretokenized': b'JAVA_BLOCK:public Repository getRepository(String uid, Integer maxUsers) throws GreenPepperServerException { try { sessionService.startSession(); Repository repository = loadRepository(uid); if (maxUsers != null) { repository.setMaxUsers(maxUsers); } return repository; } finally <extra_id_0> }', 'inputs': array([    3,  7641,    15,  3517,    56,  4569,  2522, 11250,     5,
          31,  4103,     9,   237,   350,  2178,     8,    42,     3,
       11541,   364,  4384,  3238, 12598,     7,    93,     7,   509,
         108,     4,   373,   413,    18,  2522,  1576,    11,   551,
         458,     5,  3477,    10,    21,    17,   532,  2178,    49,
          30,     8,     7,  1576,     4,  5690,  2178,     5,   532,
        2178,    10,     6,    14,  1576,    13,     6,   658, 32099,
           6,     1], dtype=int32), 'targets_pretokenized': b'{ sessionService.closeSession(); }', 'targets': array([  7, 509, 108,   4, 

ANDROID CONSTRUCT

In [ ]:
def nq_android_construct(split, shuffle_files=True):
  # We only have one file for each split.
  del shuffle_files

   # Load lines from the text file as examples.

  ds = tf.data.TextLineDataset(nq_tsv_path_android_construct[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

print("A few raw train examples...")
for ex in tfds.as_numpy(nq_android_construct("train").take(5)):
  print(ex)

A few raw train examples...
{'input': b'private void writeToFile(final String content) { if ( <extra_id_0>) { return; } try { System.err.println(content); File f = new File(OUTPUT_FILE); f.delete(); FileWriter fstream = new FileWriter(OUTPUT_FILE); BufferedWriter out = new BufferedWriter(fstream); out.write(content); out.close(); } catch (Exception e) { e.printStackTrace(); } }', 'output': b'!WRITE_TO_FILE'}
{'input': b'private void writeToFile(final String content) { if (!WRITE_TO_FILE) { return; } try { System.err.println(content); File f = new File(OUTPUT_FILE); f.delete(); FileWriter fstream = new FileWriter(OUTPUT_FILE); BufferedWriter out = new BufferedWriter(fstream); out.write(content); out.close(); } catch ( <extra_id_0>) { e.printStackTrace(); } }', 'output': b'Exception e'}
{'input': b'private void writeToFile(final String content) { if (!WRITE_TO_FILE) { return; } try { System.err.println(content); File f = new File( <extra_id_0>); f.delete(); FileWriter fstream = new FileW

In [ ]:
def android_construct_preprocessing(ds):
  
  def to_inputs_and_targets(ex):

        inputs = tf.strings.join(['ANDROID_CONSTRUCT:' + ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label }
    
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
t5.data.TaskRegistry.remove('android_construct')
t5.data.TaskRegistry.add(
    "android_construct",
    dataset_fn=nq_android_construct,
    splits=["train", "validation"],
    text_preprocessor=[android_construct_preprocessing],
    output_features = DEFAULT_OUTPUT_FEATURES,
    metric_fns=[t5.evaluation.metrics.accuracy],
    num_input_examples=num_nq_examples_android_construct
)

In [ ]:
nq_task = t5.data.TaskRegistry.get("android_construct")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": 256, "targets": 256})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)


/usr/local/lib/python3.6/dist-packages/t5/seqio/preprocessors.py:65: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  _tokenize, num_parallel_calls=tf.data.experimental.AUTOTUNE)


A few preprocessed training examples...
{'inputs_pretokenized': b'ANDROID_CONSTRUCT:public void onTimeSet(TimePicker view, int hourOfDay, int minute) { editor = prefs.edit(); Log.i("CfDC", "Hour: " + hourOfDay + " Minute: " + minute); editor.putInt( <extra_id_0>); editor.putInt(baseExtensionSettings.getPrefTimeMinute(), minute); editor.commit(); }', 'inputs': array([    3, 16446,    15,  5071, 17558,    56,  4569,    20,   170,
         199,   174,     5, 20635,   403,     9,    35, 26952,     9,
          35,  8569,     8,     7,  1424,    11,  2344,     4,  1769,
          18,   319,     4,    86,    28,   302,   165,  5691,    43,
          32,  5705,    56,    32,    34, 26952,    34,    32,     3,
        7130,    56,    32,    34,  8569,    10,  1424,     4,  2870,
           5, 32099,    10,  1424,     4,  2870,     5,  1015,   867,
         460,     4,    33,  2503,   199,  7130,    72,  8569,    10,
        1424,     4,  1827,    18,     6,     1], dtype=int32), 'targets_preto

ANDROID TOKEN

In [ ]:
def nq_android_token(split, shuffle_files=False):
  # We only have one file for each split.
  del shuffle_files

  # Load lines from the text file as examples.
  ds = tf.data.TextLineDataset(nq_tsv_path_android_token[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

print("A few raw valid examples...")
for ex in tfds.as_numpy(nq_android_token("validation").take(5)):
  print(ex)

A few raw valid examples...
{'input': b'public void register(Context context <extra_id_0> IntentFilter intentFilter = buildFilter(); try { register(context, intentFilter); }catch (IllegalArgumentException e){ Log.w(TAG, "Error registering receiver. Receiver maybe already registered, unregistering to register again."); unregister(context); register(context, intentFilter); } }', 'output': b') {'}
{'input': b'public void register(Context context) { IntentFilter intentFilter <extra_id_0> try { register(context, intentFilter); }catch (IllegalArgumentException e){ Log.w(TAG, "Error registering receiver. Receiver maybe already registered, unregistering to register again."); unregister(context); register(context, intentFilter); } }', 'output': b'= buildFilter();'}
{'input': b'public void register(Context context) { IntentFilter intentFilter = buildFilter(); try <extra_id_0> register(context, intentFilter); }catch (IllegalArgumentException e){ Log.w(TAG, "Error registering receiver. Receiver ma

In [ ]:
def android_token_preprocessing(ds):
  
  def to_inputs_and_targets(ex):

        inputs = tf.strings.join(['ANDROID_TOKEN:' + ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label }
    
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
t5.data.TaskRegistry.remove('android_token')
t5.data.TaskRegistry.add(
    "android_token",
    dataset_fn=nq_android_token,
    splits=["train", "validation"],
    text_preprocessor=[android_token_preprocessing],
    output_features = DEFAULT_OUTPUT_FEATURES,
    metric_fns=[t5.evaluation.metrics.accuracy],
    num_input_examples=num_nq_examples_android_token
)

In [ ]:
nq_task = t5.data.TaskRegistry.get("android_token")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": 256, "targets": 256})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)


/usr/local/lib/python3.6/dist-packages/t5/seqio/preprocessors.py:65: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  _tokenize, num_parallel_calls=tf.data.experimental.AUTOTUNE)


A few preprocessed training examples...
{'inputs_pretokenized': b'ANDROID_TOKEN:protected Fragment getCurrentFrag() { List<Fragment> fragments = getListFrag(); if (fragments != null) { for (Fragment fragment : fragments <extra_id_0> if (fragment != null && fragment.isVisible()) { return fragment; } } } return null; }', 'inputs': array([    3, 16446,    15,  2591,    56, 18728,  7272,  1873, 18843,
          16,     7,    85,    25,   619,    29, 14332,    11,  7240,
       18843,    18,    21,    17,  2604,    22,    49,    30,     8,
           7,    50,    17,   619,  1691,    58, 14332, 32099,    21,
          17,  2604,    49,    30,    91,  1691,     4,  6000,    60,
           7,    14,  1691,    13,     6,     6,     6,    14,    30,
          13,     6,     1], dtype=int32), 'targets_pretokenized': b') {', 'targets': array([3, 8, 7, 1], dtype=int32)}
{'inputs_pretokenized': b'ANDROID_TOKEN:public View findViewById(int id) { if (mCachedViews.containsKey(id)) { return mCachedView

ANDROID BLOCK

In [ ]:
def nq_android_block(split, shuffle_files=False):
  # We only have one file for each split.
  del shuffle_files

  # Load lines from the text file as examples.
  ds = tf.data.TextLineDataset(nq_tsv_path_android_block[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

print("A few raw valid examples...")
for ex in tfds.as_numpy(nq_android_block("validation").take(5)):
  print(ex)

A few raw valid examples...
{'input': b'public void update() { if(Gdx.input.isKeyPressed(Keys.LEFT))<extra_id_0> if(Gdx.input.isKeyPressed(Keys.RIGHT)){ bucket.x += 200 * Gdx.graphics.getDeltaTime(); } if (bucket.x < 0) bucket.x = 0; if (bucket.x > 320 - 64) bucket.x = 320 - 64; }', 'output': b'{ bucket.x -= 200 * Gdx.graphics.getDeltaTime(); }'}
{'input': b'public void update() { if(Gdx.input.isKeyPressed(Keys.LEFT)){ bucket.x -= 200 * Gdx.graphics.getDeltaTime(); } if(Gdx.input.isKeyPressed(Keys.RIGHT))<extra_id_0> if (bucket.x < 0) bucket.x = 0; if (bucket.x > 320 - 64) bucket.x = 320 - 64; }', 'output': b'{ bucket.x += 200 * Gdx.graphics.getDeltaTime(); }'}
{'input': b'public void update() { if(Gdx.input.isKeyPressed(Keys.LEFT)){ bucket.x -= 200 * Gdx.graphics.getDeltaTime(); } if(Gdx.input.isKeyPressed(Keys.RIGHT)){ bucket.x += 200 * Gdx.graphics.getDeltaTime(); } if (bucket.x < 0) <extra_id_0> if (bucket.x > 320 - 64) bucket.x = 320 - 64; }', 'output': b'bucket.x = 0;'}
{'input':

In [ ]:
def android_block_preprocessing(ds):
  
  def to_inputs_and_targets(ex):

        inputs = tf.strings.join(['ANDROID_BLOCK:' + ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label }
    
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
t5.data.TaskRegistry.remove('android_block')
t5.data.TaskRegistry.add(
    "android_block",
    dataset_fn=nq_android_block,
    splits=["train", "validation"],
    text_preprocessor=[android_block_preprocessing],
    output_features = DEFAULT_OUTPUT_FEATURES,
    metric_fns=[t5.evaluation.metrics.accuracy],
    num_input_examples=num_nq_examples_android_block
)

In [ ]:
nq_task = t5.data.TaskRegistry.get("android_block")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": 256, "targets": 256})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)


/usr/local/lib/python3.6/dist-packages/t5/seqio/preprocessors.py:65: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  _tokenize, num_parallel_calls=tf.data.experimental.AUTOTUNE)


A few preprocessed training examples...
{'inputs_pretokenized': b'ANDROID_BLOCK:private void cancelNotification(final BluetoothDevice device) { final NotificationManager nm = (NotificationManager) getSystemService(NOTIFICATION_SERVICE); if (Build.VERSION.SDK_INT >= Build.VERSION_CODES.O) <extra_id_0> else { nm.cancel(device.getAddress(), NOTIFICATION_ID); } }', 'inputs': array([    3, 16446,    15,  3517,    56,  8797,    20,  2058,   836,
           5,    64, 14114,   979,     8,     7,    44,  1895,   121,
       11464,    11,    17,   836,   121,     8,     3,  2685,     5,
        4684,    15,  1455,    10,    21,    17,  1042,     4,  1235,
           4,  4090,    15,  2037,   453,  3080,     4,  1235,    15,
        4858,     4,   935,     8, 32099,    77,     7, 11464,     4,
        1753,     5,  1529,     4,  3569,    72,     3,  4684,    15,
         142,    10,     6,     6,     1], dtype=int32), 'targets_pretokenized': b'{ stopForeground(true); }', 'targets': array([   7,  

### Hyper Parameter tuning
You can run the HP tuning using the following cells.  
Please set the correct path of the variable *MODEL_DIR* (the path to save the pretrained model in), *PATH_GIN_FILE* (the gin file configuration for this HP tuning) and *PRETRAINED_DIR* (the folder that contains the pretrained model).  
**Keep attention** to change the *pretrained_model_dir* in finetune step (if you are starting the HP tuning from scratch you have to set the value *PRETRAINED_DIR*, if you are restarting the HP tuning from a previous saved checkpoint you have to set the value *MODEL_DIR*)

In [ ]:
def _rate_num_input_examples(task):
  if "train" in task.splits:
    return float(task.num_input_examples("train"))
  elif "validation" in task.splits:
    return float(task.num_input_examples("validation"))
  else:
    raise ValueError("Task %s does not have a train or validation split." % (task.name))


t5.data.MixtureRegistry.remove("all_tasks")
t5.data.MixtureRegistry.add(
    "all_tasks",
    ["java_construct", "java_token", "java_block", "android_construct", "android_token", "android_block"],
    default_rate=_rate_num_input_examples
     #default_rate=1.0
)

In [ ]:
from mesh_tensorflow.transformer.learning_rate_schedules import slanted_triangular

MODEL_SIZE = "small" 

# Set the folder where the checkpoints and all the others information will be writed
MODEL_DIR = 'gs://bucket_code_completion/T5_extension/HP_TUNING/constant/model'

# Specify the pre-trained dir which must contain the pre-trained models, the operative_config.gin file and the checkpoint file as well
PRETRAINED_DIR='gs://bucket_code_completion/T5_extension/pretrained_with_masking'


model_parallelism, train_batch_size, keep_checkpoint_max = {
    "small": (1, 256, 16),
    "base": (2, 128, 8),
    "large": (8, 64, 4),
    "3B": (8, 16, 1),
    "11B": (8, 16, 1)}[MODEL_SIZE]

tf.io.gfile.makedirs(MODEL_DIR)

model = t5.models.MtfModel(
    model_dir=MODEL_DIR,
    tpu=TPU_ADDRESS,
    tpu_topology=TPU_TOPOLOGY,
    model_parallelism=model_parallelism,
    batch_size=train_batch_size,
    learning_rate_schedule = 0.001,
    sequence_length={"inputs": 256, "targets": 256},
    save_checkpoints_steps=5000,
    keep_checkpoint_max=keep_checkpoint_max if ON_CLOUD else None,
    iterations_per_loop=100,
)

In [ ]:
PATH_GIN_FILE = 'gs://bucket_code_completion/T5_extension/HP_TUNING/constant/operative_config.gin'
import gin

with gin.unlock_config():
    gin.parse_config_file(PATH_GIN_FILE)
    #RUN FINE-TUNING
    FINETUNE_STEPS = 100000
    model.finetune(
        mixture_or_task_name="all_tasks",
        pretrained_model_dir=PRETRAINED_DIR,
        finetune_steps=FINETUNE_STEPS
    )

INFO:root:system_path_file_exists:gs://bucket_comment_completion/Matteo/HP_TUNING/constant/operative_config.gin
ERROR:root:Path not found: gs://bucket_comment_completion/Matteo/HP_TUNING/constant/operative_config.gin
INFO:root:system_path_file_exists:gs://bucket_comment_completion/Matteo/pretrained_with_masking/operative_config.gin
ERROR:root:Path not found: gs://bucket_comment_completion/Matteo/pretrained_with_masking/operative_config.gin


INFO:tensorflow:Using config: {'_model_dir': 'gs://bucket_comment_completion/Matteo/HP_TUNING/constant/model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': None, '_session_config': graph_options {
  rewrite_options {
    disable_meta_optimizer: true
  }
}
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.84.84.90:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({'worker': ['10.84.84.90:8470']}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.84.84.90:8470', '_evaluation_master': 'grpc://10

/usr/local/lib/python3.6/dist-packages/t5/seqio/preprocessors.py:65: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  _tokenize, num_parallel_calls=tf.data.experimental.AUTOTUNE)


INFO:tensorflow:num_cores_per_replica: 1
INFO:tensorflow:computation_shape: [1, 1, 1, 1]
INFO:tensorflow:num_replicas: 8
INFO:tensorflow:device_assignment.topology.device_coordinates: [[[0 0 0 0]
  [0 0 0 1]
  [1 0 0 0]
  [1 0 0 1]
  [0 1 0 0]
  [0 1 0 1]
  [1 1 0 0]
  [1 1 0 1]]]
INFO:tensorflow:device_assignment.core_assignment: [[[0 0 0 0]]

 [[0 0 0 1]]

 [[1 0 0 0]]

 [[1 0 0 1]]

 [[0 1 0 0]]

 [[0 1 0 1]]

 [[1 1 0 0]]

 [[1 1 0 1]]]
INFO:tensorflow:auto_logical_to_physical_tpu logical_shape=[8] physical_shape=[2, 2, 2]
INFO:tensorflow:auto_logical_to_physical_tpu logical_to_physical = [(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 1, 0), (1, 1, 1), (1, 0, 0), (1, 0, 1)]
INFO:tensorflow:SimdMeshImpl init: Shape[batch=8] LayoutRules{('experts', 'batch'), ('ensemble', 'ensemble'), ('vocab', 'model'), ('batch', 'batch'), ('heads', 'model'), ('d_ff', 'model')}
INFO:tensorflow:Device Assignment: <tensorflow.python.tpu.device_assignment.DeviceAssignment object at 0x7f1601563550>
INF

### Evaluate the performances

In [ ]:
# Use a larger batch size for evaluation, which requires less memory.
model.batch_size = 512
model.eval(
    mixture_or_task_name="all_tasks",
    # mixture_or_task_name="all_tasks",
    checkpoint_steps=-1 #evaluate only last checkpoint
)